In [2]:
from fasthtml.common import *
from fasthtml.jupyter import JupyUvi, HTMX
from dataclasses import dataclass
from datetime import datetime
from math import ceil

db = database('trfc.db')

app, rt = fast_app()

In [3]:
# Fake inputs
mgr_inputs = {
    'min_season': '1921',
    'max_season': '2024',
    'league_tiers': ['2', '3', '4', '5'],
    'inc_play_offs': 'Y',
    'competitions': ['Anglo-Italian Cup', "Associate Members' Cup", 'FA Cup', 'FA Trophy', "Full Members' Cup", 'League Cup', 'War League'],
    'pens_as_draw': 'Y',
    'venues': ['H', 'A', 'N'],
    'min_games': '10'
}

In [4]:
def validate_inputs(inputs):
    if 'league_tiers' not in inputs and 'competitions' not in inputs:
        return False
    elif 'venues' not in inputs:
        return False
    else:
        return True
    
validate_inputs(mgr_inputs)

True

In [9]:
def run_query(inputs):
    query = """
        SELECT 
            m.manager_name,
            COUNT(*) as P,
            COUNT(CASE WHEN r.outcome = 'W' THEN 1 END) as W,
            COUNT(CASE WHEN r.outcome = 'D' THEN 1 END) as D,
            COUNT(CASE WHEN r.outcome = 'L' THEN 1 END) as L,
            SUM(r.goals_for) as GF,
            SUM(r.goals_against) as GA,
            SUM(r.goals_for) - SUM(r.goals_against) as GD,
            ROUND(CAST(COUNT(CASE WHEN r.outcome = 'W' THEN 1 END) AS FLOAT) / COUNT(*) * 100, 2) as win_pc
        FROM results r
        LEFT JOIN manager_reigns mr ON r.game_date >= mr.mgr_date_from
            AND (r.game_date <= mr.mgr_date_to OR mr.mgr_date_to IS NULL)
        LEFT JOIN managers m ON mr.manager_id = m.manager_id
        LEFT JOIN league_tiers lt ON r.season = lt.season AND r.game_type = 'League'
        WHERE m.manager_name = 'Nigel Adkins'
        GROUP BY m.manager_name
        ORDER BY P DESC
    """

    return db.conn.execute(query)

In [10]:
def get_col_names(data):
    return [x[0] for x in data.description]

In [11]:
def get_data(data):
    return data.fetchall()

In [12]:
data = run_query(mgr_inputs)
col_names = get_col_names(data)
records = get_data(data)

In [13]:
import pandas as pd
df = pd.DataFrame(records, columns=col_names)
df.tail()

,manager_name,P,W,D,L,GF,GA,GD,win_pc
0,Nigel Adkins,69,24,13,32,88.0,105.0,-17.0,34.78


In [23]:
@dataclass
class ManagerRecord:
    name: str
    P: int
    W: int
    D: int
    L: int
    GF: int
    GA: int
    GD: int
    win_pc: float

In [22]:
seasons = ['2024/25', '2023/24', '2022/23']
placeholders = ",".join(["?"] * len(seasons))

params = list(seasons)

ssn_query = f"""
    SELECT 
        r.season,
        COUNT(*) as P,
        COUNT(CASE WHEN r.outcome = 'W' THEN 1 END) as W,
        COUNT(CASE WHEN r.outcome = 'D' THEN 1 END) as D,
        COUNT(CASE WHEN r.outcome = 'L' THEN 1 END) as L,
        SUM(r.goals_for) as GF,
        SUM(r.goals_against) as GA,
        SUM(r.goals_for) - SUM(r.goals_against) as GD,
        ROUND(CAST(COUNT(CASE WHEN r.outcome = 'W' THEN 1 END) AS FLOAT) / COUNT(*) * 100, 2) as win_pc,
        COUNT(CASE WHEN r.game_type = 'League' THEN 1 END) as league_games,
        (COUNT(CASE WHEN r.game_type = 'League' AND r.outcome = 'W' THEN 1 END) * 3 + 
        COUNT(CASE WHEN r.game_type = 'League' AND r.outcome = 'D' THEN 1 END)) as league_pts,
        CASE 
            WHEN COUNT(CASE WHEN r.game_type = 'League' THEN 1 END) > 0 
            THEN ROUND(CAST(
                (COUNT(CASE WHEN r.game_type = 'League' AND r.outcome = 'W' THEN 1 END) * 3 + 
                COUNT(CASE WHEN r.game_type = 'League' AND r.outcome = 'D' THEN 1 END)) AS FLOAT) / 
                COUNT(CASE WHEN r.game_type = 'League' THEN 1 END), 2)
            ELSE 0 
        END as league_ppg
    FROM results r
    LEFT JOIN manager_reigns mr ON r.game_date >= mr.mgr_date_from
        AND (r.game_date <= mr.mgr_date_to OR mr.mgr_date_to IS NULL)
    LEFT JOIN managers m ON mr.manager_id = m.manager_id
    LEFT JOIN league_tiers lt ON r.season = lt.season AND r.game_type = 'League'
    WHERE r.season IN ({placeholders})
    GROUP BY r.season
    ORDER BY r.season DESC
"""

db.conn.execute(ssn_query, params).fetchall()

[('2024/25', 27, 8, 6, 13, 26.0, 42.0, -16.0, 29.63, 20, 21, 1.05),
 ('2023/24', 52, 17, 8, 27, 72.0, 81.0, -9.0, 32.69, 46, 57, 1.24),
 ('2022/23', 53, 16, 16, 21, 56.0, 62.0, -6.0, 30.19, 46, 58, 1.26)]

In [ ]:
@dataclass
class SeasonRecord:
    season: str
    P: int
    W: int
    D: int
    L: int
    GF: int
    GA: int
    GD: int
    win_pc: float
    league_games: int
    league_pts: int
    league_ppg: float

In [101]:
selected_season = '2024/25'

params = [selected_season]

ssn_query = f"""
    SELECT 
        r.season,
        r.competition,
        COUNT(*) as P,
        COUNT(CASE WHEN r.outcome = 'W' THEN 1 END) as W,
        COUNT(CASE WHEN r.outcome = 'D' THEN 1 END) as D,
        COUNT(CASE WHEN r.outcome = 'L' THEN 1 END) as L,
        SUM(r.goals_for) as GF,
        SUM(r.goals_against) as GA,
        SUM(r.goals_for) - SUM(r.goals_against) as GD,
        ROUND(CAST(COUNT(CASE WHEN r.outcome = 'W' THEN 1 END) AS FLOAT) / COUNT(*) * 100, 2) as win_pc
    FROM results r
    LEFT JOIN manager_reigns mr ON r.game_date >= mr.mgr_date_from
        AND (r.game_date <= mr.mgr_date_to OR mr.mgr_date_to IS NULL)
    LEFT JOIN managers m ON mr.manager_id = m.manager_id
    LEFT JOIN season_league_tiers slt ON r.season = slt.season AND r.game_type = 'League'
    WHERE r.season = ?
    GROUP BY r.season, r.competition
    ORDER BY r.season DESC, P DESC
"""

db.conn.execute(ssn_query, params).fetchall()

[('2024/25', 'League Two', 3, 1, 2, 0, 1, 0, 1, 33.33),
 ('2024/25', 'Carabao Cup', 2, 1, 0, 1, 3, 4, -1, 50.0),
 ('2024/25', 'Bristol Street Motors Trophy', 1, 0, 0, 1, 1, 3, -2, 0.0)]

In [16]:
dataclass
class CompetitionRecord:
    season: str
    competition: str
    P: int
    W: int
    D: int
    L: int
    GF: int
    GA: int
    GD: int
    win_pc: float

In [19]:
streaks_query  = f'''
    SELECT 
        r.*
    FROM results r
    LEFT JOIN manager_reigns mr ON r.game_date >= mr.mgr_date_from
        AND (r.game_date <= mr.mgr_date_to OR mr.mgr_date_to IS NULL)
    LEFT JOIN managers m ON mr.manager_id = m.manager_id
    LEFT JOIN league_tiers lt ON r.season = lt.season AND r.game_type = 'League'
'''

# Load base data
df = pd.read_sql(streaks_query, db.conn)

# Basic stats
stats = df.groupby('season').agg({
    'outcome': ['count',
            lambda x: (x == 'W').sum(),
            lambda x: (x == 'D').sum(),
            lambda x: (x == 'L').sum()],
    'goals_for': 'sum',
    'goals_against': 'sum'
}).reset_index()

# Calculate streaks
def get_streak_lengths(group, condition):
    # Creates groups of consecutive True values and returns their lengths
    streak_groups = (group != group.shift()).cumsum()
    return group.groupby(streak_groups).sum()

# Example for win streaks
df['is_win'] = df['outcome'] == 'W'
df['is_unbeaten'] = df['outcome'] != 'L'
df['is_clean_sheet'] = df['goals_against'] == 0
df['win_to_nil'] = (df['outcome'] == 'W') & (df['goals_against'] == 0)
df['is_draw'] = df['outcome'] == 'D'
df['is_defeat'] = df['outcome'] == 'L'
df['winless'] = df['outcome'] != 'W'
df['blanks'] = df['goals_for'] == 0
df['defeat_to_nil'] = (df['outcome'] == 'L') & (df['goals_for'] == 0)


df.groupby('season').agg({
    'is_win': lambda x: get_streak_lengths(x, True).max(),
    'is_unbeaten': lambda x: get_streak_lengths(x, True).max(),
    'is_clean_sheet': lambda x: get_streak_lengths(x, True).max(),
    'win_to_nil': lambda x: get_streak_lengths(x, True).max(),
    'is_draw': lambda x: get_streak_lengths(x, True).max(),
    'is_defeat': lambda x: get_streak_lengths(x, True).max(),
    'winless': lambda x: get_streak_lengths(x, True).max(),
    'blanks': lambda x: get_streak_lengths(x, True).max(),
    'defeat_to_nil': lambda x: get_streak_lengths(x, True).max()
}).reset_index()

,season,is_win,is_unbeaten,is_clean_sheet,win_to_nil,is_draw,is_defeat,winless,blanks,defeat_to_nil
0,1921/22,2,3,2,1,2,6,15,3,3
1,1922/23,4,4,2,2,2,7,11,4,3
2,1923/24,3,11,3,2,3,4,7,3,3
3,1924/25,2,4,2,2,1,4,6,2,2
4,1925/26,6,7,2,2,1,5,6,3,2
...,...,...,...,...,...,...,...,...,...,...
99,2020/21,6,8,4,4,3,2,7,4,2
100,2021/22,4,8,3,3,3,2,5,4,2
101,2022/23,3,5,3,3,2,2,10,2,2
102,2023/24,4,6,2,1,2,7,7,4,4
